In [1]:
import sys

In [2]:
sys.path.append("/Users/diegoesparza/CS_Ventures/current_projects/chess_engine_using_probabilities/my_engine/chess-sauce")

In [3]:
import my_engine
import chess
import stockfish
import random
from line_profiler import LineProfiler
import pandas as pd

In [4]:
sf = stockfish.Stockfish(path="/Users/diegoesparza/Downloads/stockfish/stockfish-macos-x86-64")
sf.resume_full_strength()

In [5]:
# if second best piece to move can't make a move nearly as good as best move,
# probably not stable
# 
# if best non-capture/non-check move isn't nearly as good as best move, probably
# not stable.
#
# board piece eval at end should be within -200 200
# centipawn eval should be within -200 200 (this we'll track as we go along)
def position_is_stable(b, sf):
    # assume sf is set to b's position
    score, _ = my_engine.eval_piece_vals.eval_piece_vals(b)
    if abs(score) > 200:
        print(f"piece evals: {score}")
        return False
   
    eval = sf.get_evaluation(searchtime=20)
    if eval["type"] != "cp":
        print("mate")
        return False
    if abs(eval["value"]) > 200:
        print(f"eval: {eval["value"]}")
        return False
   
    moves = sf.get_top_moves(7, num_nodes=10000)
    for move in moves:
        move["Move"] = chess.Move.from_uci(move["Move"])
    top_centipawn = moves[0]["Centipawn"]
    top_origin = moves[0]["Move"].from_square
    mult = 1 if b.turn else -1
   
    # check origins, captures
    must_capture = True
    must_move_specific_piece = True
    for move in moves:
        centipawn = move["Centipawn"]
        if mult * centipawn + 200 < mult * top_centipawn:
            print(f"moves: {moves}")
            return False
           
        if not b.is_capture(move["Move"]):
            must_capture = False
        if move["Move"].from_square != top_origin:
            must_move_specific_piece = False
            
        if not (must_capture or must_move_specific_piece):
            break
    if must_capture or must_move_specific_piece:
        print(f"moves: {moves}")
    return not (must_capture or must_move_specific_piece)


def gen_positions_from(b, sf, positions_to_generate, depth):
    fen = b.fen()
    sf.set_fen_position(fen)
    if depth == 0:
        next_depth = 0
        if positions_to_generate != 1:
            raise ValueError(positions_to_generate)
        if position_is_stable(b, sf):
            print(f"stable: {fen}")
        else:
            print(f"unstable: {fen}")
        return None
    else:
        moves = sf.get_top_moves(min(7, positions_to_generate), num_nodes=10000)
        moves = [move for move in moves if abs(move["Centipawn"]) < 500]
        if len(moves) == 0:
            print(f"skewed: {fen}")
            
        moves_to_explore = [chess.Move.from_uci(move["Move"]) for move in moves]
        next_depth = depth-1
        moves = list(b.legal_moves)

    pos_per = positions_to_generate // len(moves_to_explore)
    rem = positions_to_generate - pos_per * len(moves_to_explore)
    for idx, move in enumerate(moves_to_explore):
        next_positions_to_generate = pos_per
        if idx < rem:
            next_positions_to_generate += 1
        b.push(move)
        gen_positions_from(b, sf, next_positions_to_generate, next_depth)
        b.pop()
        
def gen_positions(sf, opening_fens, positions_to_generate, depth):
    if positions_to_generate < len(opening_fens):
        opening_fens = random.sample(opening_fens, positions_to_generate)
        
    pos_per = positions_to_generate // len(opening_fens)
    rem = positions_to_generate - pos_per * len(opening_fens)
    for idx, fen in enumerate(opening_fens):
        next_positions_to_generate = pos_per
        if idx < rem:
            next_positions_to_generate += 1
        b = chess.Board(fen)
        try:
            gen_positions_from(b, sf, next_positions_to_generate, depth)
        except ValueError as e:
            print(f"Error from {fen}: {e}")

In [6]:
positions_to_generate = 10
depth = 8
random.seed(0)

lp = LineProfiler()
lp.add_function(gen_positions_from)
lp.add_function(gen_positions)
lp.add_function(position_is_stable)
opening_fens = !cat "/Users/diegoesparza/CS_Ventures/current_projects/chess_engine_using_probabilities/kaggle_fens/strong_openings"
lp.runcall(gen_positions, sf, opening_fens, positions_to_generate, depth)
lp.print_stats()

stable: r2q1rk1/4b1pp/2p5/p1PpP3/3Qp3/8/PP3PPP/R1B2RK1 w - - 0 17
stable: r2q1r2/ppp1nppk/3p1n1p/4p3/P1BPP1Q1/2N1P2P/1PP3P1/R4RK1 w - - 3 15
piece evals: -265.125
unstable: r2q1rk1/p1ppbppp/1p2p3/n3P3/3P1P1P/P1Q1BN2/1PP3P1/2KR1b1R w - - 0 15
stable: 2rq1r1k/1b1pnpbp/p5p1/1p1Np3/1P1nP3/P1N5/BB3PPP/R2Q1RK1 w - - 3 17
piece evals: -293.375
unstable: r1bq1rk1/2p3p1/1pn1pp2/3pP2Q/3P3n/P1P5/5PPP/R1B1KB1R w KQ - 2 15
stable: r3kb1r/pp3ppp/8/8/2Bn2b1/6P1/PPPB1PP1/R3K1NR w KQkq - 1 13
moves: [{'Move': Move.from_uci('f8h6'), 'Centipawn': -163, 'Mate': None}, {'Move': Move.from_uci('f8e7'), 'Centipawn': -205, 'Mate': None}, {'Move': Move.from_uci('f3e5'), 'Centipawn': -435, 'Mate': None}, {'Move': Move.from_uci('a2f7'), 'Centipawn': -443, 'Mate': None}, {'Move': Move.from_uci('d3d4'), 'Centipawn': -486, 'Mate': None}, {'Move': Move.from_uci('c3b5'), 'Centipawn': -530, 'Mate': None}, {'Move': Move.from_uci('c3a4'), 'Centipawn': -559, 'Mate': None}]
unstable: rnbq1Bk1/4bp1p/2pp4/1p2p2p/p3P3/P1NP1N2

In [22]:
sf.set_fen_position("Bn1qbrkb/p2n1p1p/6p1/3p4/Q2P3P/4PN2/PP2KPP1/R1B4R b - - 0 14", send_ucinewgame_token=True)
for i in range(10):
    print(sf.get_top_moves(5, num_nodes=10000))

[{'Move': 'd7c5', 'Centipawn': -147, 'Mate': None}, {'Move': 'd7b6', 'Centipawn': -189, 'Mate': None}, {'Move': 'd7e5', 'Centipawn': -151, 'Mate': None}, {'Move': 'h8f6', 'Centipawn': -205, 'Mate': None}, {'Move': 'd8c8', 'Centipawn': -211, 'Mate': None}]
[{'Move': 'd7c5', 'Centipawn': -120, 'Mate': None}, {'Move': 'd7b6', 'Centipawn': -193, 'Mate': None}, {'Move': 'd8c7', 'Centipawn': -193, 'Mate': None}, {'Move': 'h8f6', 'Centipawn': -217, 'Mate': None}, {'Move': 'd8c8', 'Centipawn': -222, 'Mate': None}]
[{'Move': 'd7c5', 'Centipawn': -92, 'Mate': None}, {'Move': 'a7a6', 'Centipawn': -173, 'Mate': None}, {'Move': 'd7b6', 'Centipawn': -190, 'Mate': None}, {'Move': 'h8g7', 'Centipawn': -193, 'Mate': None}, {'Move': 'd8c7', 'Centipawn': -201, 'Mate': None}]
[{'Move': 'd7c5', 'Centipawn': -69, 'Mate': None}, {'Move': 'd7b6', 'Centipawn': -191, 'Mate': None}, {'Move': 'a7a6', 'Centipawn': -208, 'Mate': None}, {'Move': 'h8f6', 'Centipawn': -208, 'Mate': None}, {'Move': 'h8g7', 'Centipawn':

In [51]:
sf.get_parameters()

{'Debug Log File': '',
 'Contempt': 0,
 'Min Split Depth': 0,
 'Ponder': 'false',
 'MultiPV': 1,
 'Skill Level': 20,
 'Move Overhead': 2,
 'Minimum Thinking Time': 4,
 'Slow Mover': 5,
 'UCI_Chess960': 'false',
 'UCI_LimitStrength': 'true',
 'UCI_Elo': 3000,
 'Threads': 1,
 'Hash': 16}

In [6]:
sf.get_top_moves?

Signature:
sf.get_top_moves(
    num_top_moves: 'int' = 5,
    verbose: 'bool' = False,
    num_nodes: 'int' = 0,
) -> 'List[dict]'
Docstring:
Returns info on the top moves in the position.

Args:
    num_top_moves:
      The number of moves for which to return information, assuming there
      are at least that many legal moves.
      Default is 5.

    verbose:
      Option to include the full info from the engine in the returned dictionary,
      including seldepth, multipv, time, nodes, nps, and wdl if available.
      `Boolean`. Default is `False`.

    num_nodes:
      Option to search until a certain number of nodes have been searched, instead of depth.
      Default is 0.

Returns:
    A list of dictionaries, where each dictionary contains keys for `Move`, `Centipawn`, and `Mate`.
    The corresponding value for either the `Centipawn` or `Mate` key will be `None`.
    If there are no moves in the position, an empty list is returned.

    If `verbose` is `True`, the dictionary w

In [47]:
sf?

Type:        Stockfish
String form: <stockfish.models.Stockfish object at 0x1267a1450>
File:        ~/.pyenv/versions/3.13.0/lib/python3.13/site-packages/stockfish/models.py
Docstring:   Integrates the Stockfish chess engine with Python.

In [57]:
sf.get_best_move(wtime=5, btime=5)

'd7b6'

In [13]:
sf.set_fen_position?

Signature:
sf.set_fen_position(
    fen_position: 'str',
    send_ucinewgame_token: 'bool' = True,
) -> 'None'
Docstring:
Sets current board position in Forsyth-Edwards notation (FEN).

Args:
    fen_position:
      FEN string of board position.

    send_ucinewgame_token:
      Whether to send the `ucinewgame` token to the Stockfish engine.
      The most prominent effect this will have is clearing Stockfish's transposition table,
      which should be done if the new position is unrelated to the current position.

Returns:
    `None`

Example:
    >>> stockfish.set_fen_position("1nb1k1n1/pppppppp/8/6r1/5bqK/6r1/8/8 w - - 2 2")
File:      ~/CS_Ventures/current_projects/chess_engine_using_probabilities/stockfish_python_wrapper/stockfish/stockfish/models.py
Type:      method

In [14]:
sf.get_evaluation?

Signature: sf.get_evaluation(searchtime: 'Optional[int]' = None) -> 'Dict[str, Union[str, int]]'
Docstring:
Searches to the specified depth and evaluates the current position.

Args:
    searchtime:
      [Optional] Time for Stockfish to evaluate in milliseconds (int)

Returns:
    A dictionary of two pairs: {str: str, str: int}
    - The first pair describes the type of the evaluation. The key is "type", and the value
      will be either "cp" (centipawns) or "mate".
    - The second pair describes the value of the evaluation. The key is "value", and the value
      will be an int (representing either a cp value or a mate in n value).
File:      ~/CS_Ventures/current_projects/chess_engine_using_probabilities/stockfish_python_wrapper/stockfish/stockfish/models.py
Type:      method